In [41]:
import os
import chardet
from pathlib import Path
import pyeeg

import mne
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline

from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, StackingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import ExtraTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

Worked on after eeg_motor_imagery_nn.ipynb to extract various features and then model. 

Changes needed:
edit 27th July, 2022
1. for each of the epochs break into various bands [break into 5 bands alpha, beta...] start calculating the features
2. features: psd, attention (alpha band/beta band), frontal assymetery (difference between the left hemisphere and right)
3. average of time_series_data (just take the average of epoch)


Sources used:
https://mne.tools/stable/generated/mne.time_frequency.psd_welch.html (visited: 05/08/2022)
https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#sphx-glr-auto-tutorials-time-freq-20-sensors-time-frequency-py (visited: 05/08/2022)

In [2]:
def load_subjects_task(subject, task, run_number_left = 0, run_number_right = 3):
    event_id = dict(hands=2, feet=3)
    runs = [[3, 7, 11], [4, 8, 12], [5, 9, 13], [6, 10, 14]]   
    # runs[0] open and close left or right fist
    # runs[1] imagine opening and closing left or right fist
    # runs[2] open and close both fists or both feet
    # runs[3] imagine opening and closing both fists or both feet

    """
        T0 corresponds to rest
        T1 corresponds to onset of motion (real or imagined) of
            the left fist (in runs 3, 4, 7, 8, 11, and 12)
            both fists (in runs 5, 6, 9, 10, 13, and 14)
        T2 corresponds to onset of motion (real or imagined) of
            the right fist (in runs 3, 4, 7, 8, 11, and 12)
            both feet (in runs 5, 6, 9, 10, 13, and 14)
    """
    raw_fnames = mne.datasets.eegbci.load_data(subject, runs[task][run_number_left:run_number_right])
    raw = mne.io.concatenate_raws([mne.io.read_raw_edf(f, preload=True) for f in raw_fnames])
    channel_rename_dict = {'Fc5.':'FC5', 'Fc3.':'FC3', 'Fc1.':'FC1', 'Fcz.':'FCz', 
                      'Fc2.':'FC2', 'Fc4.':'FC4', 'Fc6.':'FC6', 'C5..':'C5', 
                      'C3..':'C3', 'C1..':'C1', 'Cz..':'Cz', 'C2..':'C2', 
                      'C4..':'C4', 'C6..':'C6', 'Cp5.':'CP5', 'Cp3.':'CP3', 
                      'Cp1.':'CP1', 'Cpz.':'CPz', 'Cp2.':'CP2', 'Cp4.':'CP4', 
                      'Cp6.':'CP6', 'Fp1.':'Fp1', 'Fpz.':'Fpz', 'Fp2.':'Fp2', 
                      'Af7.':'AF7', 'Af3.':'AF3', 'Afz.':'AFz', 'Af4.':'AF4', 
                      'Af8.':'AF8', 'F7..':'F7', 'F5..':'F5', 'F3..':'F3', 
                      'F1..':'F1', 'Fz..':'Fz', 'F2..':'F2', 'F4..':'F4', 
                      'F6..':'F6', 'F8..':'F8', 'Ft7.':'FT7', 'Ft8.':'FT8', 
                      'T7..':'T7', 'T8..':'T8', 'T9..':'T9', 'T10.':'T10', 
                      'Tp7.':'TP7', 'Tp8.':'TP8', 'P7..':'P7', 'P5..':'P5', 
                      'P3..':'P3', 'P1..':'P1', 'Pz..':'Pz', 'P2..':'P2', 
                      'P4..':'P4', 'P6..':'P6', 'P8..':'P8', 'Po7.':'PO7', 
                      'Po3.':'PO3', 'Poz.':'POz', 'Po4.':'PO4', 'Po8.':'PO8', 
                      'O1..':'O1', 'Oz..':'Oz', 'O2..':'O2', 'Iz..':'Iz'}
    raw.rename_channels(channel_rename_dict)
    montage = mne.channels.make_standard_montage('standard_1005')
    raw.set_montage(montage)
    events, _ = mne.events_from_annotations(raw, event_id=dict(T0=1, T1=2, T2=3))
    return raw, events

In [3]:
def fft(data, fs, plotting = 0):
    """
    Parameters
    ----------
    data : is 1d list/np.array
        of the signal
    fs : int
        sampling frequency
    plotting : int | 0 or 1
        plots the eeg band values
    
    Returns
    -------
    eeg_band_fft : dict 
        mean of power of signal in that band
    """
    # data is 1d
    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(data))
    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    
    # Define EEG bands
    eeg_bands = {'delta': (0, 4),
                 'theta': (4, 8),
                 'alpha': (8, 12),
                 'beta1': (12, 20),
                 'beta2': (20, 30),
                 'gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    # TODO: doubt should I take mean of this or not
    eeg_band_fft, eeg_band_fft_mean = {}, {}
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = fft_vals[freq_ix]
        eeg_band_fft_mean[band] = np.mean(fft_vals[freq_ix])
    
    # print("PRINTING EEG_BAND_FFT", eeg_band_fft)
    # Plot the data (using pandas here cause it's easy)
    if plotting == 1:
        df = pd.DataFrame(columns=['band', 'val'])
        df['band'] = eeg_band_fft_mean.keys()
        df['val'] = [eeg_band_fft_mean[band] for band in eeg_band_fft_mean]
        for band in eeg_band_fft_mean:
            print(eeg_band_fft_mean[band])
        ax = df.plot.bar(x='band', y='val', legend=False)
        ax.set_xlabel("EEG band")
        ax.set_ylabel("Mean band Amplitude")
    return eeg_band_fft, eeg_band_fft_mean

In [4]:
def preprocess_raw_data_2d_extract_features(raw, events, participant_id, df_eeg_features, plotting=0):
    # edit 27th July, 2022
    # for each of the epochs break into various bands [break into 5 bands alpha, beta...]
    # start calculating the features
    # features: psd, attention (alpha band/beta band), frontal assymetery (difference between the left hemisphere and right)
    # average of time_series_data (just take the average of epoch)
    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')
    event_id = dict(hands=2, feet=3)
    tmin, tmax = -1., 4.
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False)
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2
    i = 0
    for epoch in epochs_train:
        
        # bands
        eeg_band_fft, eeg_band_fft_mean = fft(epoch, epochs.info['sfreq'])
        
        data = eeg_band_fft_mean
        data["label"] = labels[i]
        data["participant_id"] = participant_id
        
        # epoch average
        epoch_avg = np.average(epoch)
        data["epoch_avg"] = epoch_avg
        # psd
        # TODO: DOUBT how to add this to dataset
        # what should n_fft be: 161, 100, or 99
        """
        psds_welch_mean, freqs_mean = {}, {}
        for key, value in eeg_band_fft.items():
            print(eeg_band_fft[key].shape)
            raw_array = mne.io.RawArray(eeg_band_fft[key], epochs.info)
            psds_welch_mean[key], freqs_mean[key] = mne.time_frequency.psd_welch(raw_array, n_fft=99, average='mean')
            psds_welch_median[key], freqs_median[key] = mne.time_frequency.psd_welch(raw_array, n_fft=99, average='median')
            print("PRINTING PSD WELCH SHAPE", psds_welch_mean[key].shape)
            data["psd_welch" + key] = np.mean(psds_welch_mean[key])
            data["freq" + key] = np.mean(freqs_mean[key])
        """
        
        eeg_bands = {'delta': (0, 4),
                     'theta': (4, 8),
                     'alpha': (8, 12),
                     'beta1': (12, 20),
                     'beta2': (20, 30),
                     'gamma': (30, 45)}
        
        psds_welch_mean, freqs_mean = {}, {}
        psds_welch_median, freqs_median = {}, {}
        for key, value in eeg_bands.items():
            kwargs = dict(fmin=value[0], fmax=value[1])
            psds_welch_mean[key], freqs_mean[key] = mne.time_frequency.psd_welch(epochs_train[i], n_fft=99, average='mean', **kwargs)
            psds_welch_median[key], freqs_median[key] = mne.time_frequency.psd_welch(epochs_train[i], n_fft=99, average='median', **kwargs)
            data["psd_mean_" + key] = np.mean(psds_welch_mean[key])
            data["psd_median_" + key] = np.mean(psds_welch_median[key])
            
        if plotting == 1:
            plot_for = "gamma"
            # We will only plot the PSD for a single sensor in the first epoch.
            ch_name = 'Fp1'
            ch_idx = epochs.info['ch_names'].index(ch_name)
            epo_idx = 0
            _, ax = plt.subplots()
            ax.plot(freqs_mean[plot_for], psds_welch_mean[plot_for][epo_idx, ch_idx, :], color='k',
                    ls='-', label='mean of segments')
            ax.plot(freqs_median[plot_for], psds_welch_median[plot_for][epo_idx, ch_idx, :], color='k',
                    ls='--', label='median of segments')

            ax.set(title='Welch PSD ({}, Epoch {})'.format(ch_name, epo_idx),
                   xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
            ax.legend(loc='upper right')
            plt.show()

        
        df_eeg_features = df_eeg_features.append(data, ignore_index=True)
        i += 1
    return df_eeg_features

In [5]:
def load_participant_data_extract_features(num_of_participants = 109):
    training_data_of_participants = []
    labels_of_participants = []
    df_eeg_features = pd.DataFrame()
    for i in range(1, num_of_participants + 1):
        subject_raw, subject_events = load_subjects_task(i, 0)
        df_eeg_features = preprocess_raw_data_2d_extract_features(subject_raw, subject_events, i, df_eeg_features, plotting = 0)
    return df_eeg_features

In [57]:
df_bands = load_participant_data_extract_features(15)

Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandp

/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


...

/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective wind

/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective wind

/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(data, ignore_index=True)
/tmp/ipykernel_3732/4015867031.py:77: Future

In [58]:
df_bands["label"] = df_bands["label"].astype('int')
df_bands

,delta,theta,alpha,beta1,beta2,gamma,label,participant_id,epoch_avg,psd_mean_delta,...,psd_mean_theta,psd_median_theta,psd_mean_alpha,psd_median_alpha,psd_mean_beta1,psd_median_beta1,psd_mean_beta2,psd_median_beta2,psd_mean_gamma,psd_median_gamma
0,0.000108,0.000102,0.000101,0.000101,0.000101,0.000100,1,1.0,2.600993e-07,3.751744e-14,...,1.000583e-11,1.000583e-11,3.251882e-11,3.251882e-11,3.079336e-11,3.079336e-11,7.802206e-12,7.802206e-12,8.085821e-13,8.085821e-13
1,0.000137,0.000132,0.000121,0.000119,0.000127,0.000129,0,1.0,1.896410e-07,2.644563e-14,...,5.331626e-12,5.331626e-12,1.268858e-11,1.268858e-11,5.954426e-12,5.954426e-12,6.656528e-12,6.656528e-12,5.310600e-13,5.310600e-13
2,0.000125,0.000127,0.000119,0.000113,0.000131,0.000130,0,1.0,-2.781535e-07,3.071817e-14,...,8.286917e-12,8.286917e-12,7.894770e-12,7.894770e-12,6.250401e-12,6.250401e-12,6.599638e-12,6.599638e-12,4.842216e-13,4.842216e-13
3,0.000135,0.000138,0.000139,0.000125,0.000128,0.000123,1,1.0,-1.403661e-07,4.574630e-14,...,7.374908e-12,7.374908e-12,3.346316e-11,3.346316e-11,2.533117e-11,2.533117e-11,8.326582e-12,8.326582e-12,1.080379e-12,1.080379e-12
4,0.000132,0.000138,0.000139,0.000129,0.000136,0.000143,1,1.0,2.001740e-07,5.192557e-14,...,4.859422e-12,4.859422e-12,2.251596e-11,2.251596e-11,2.259193e-11,2.259193e-11,7.387809e-12,7.387809e-12,9.612132e-13,9.612132e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,0.000249,0.000254,0.000243,0.000229,0.000235,0.000223,0,15.0,-2.362602e-07,3.465410e-13,...,7.782590e-12,7.782590e-12,5.714636e-11,5.714636e-11,3.154895e-11,3.154895e-11,1.454001e-11,1.454001e-11,2.521980e-12,2.521980e-12
671,0.000255,0.000241,0.000222,0.000234,0.000227,0.000244,1,15.0,-6.570528e-07,4.651445e-13,...,2.641615e-11,2.641615e-11,1.145377e-10,1.145377e-10,4.798707e-11,4.798707e-11,1.237254e-11,1.237254e-11,1.852608e-12,1.852608e-12
672,0.000282,0.000287,0.000278,0.000252,0.000271,0.000272,1,15.0,-3.912246e-07,5.610510e-13,...,1.316276e-10,1.316276e-10,1.393927e-10,1.393927e-10,5.533103e-11,5.533103e-11,9.233851e-12,9.233851e-12,1.328362e-12,1.328362e-12
673,0.000261,0.000274,0.000263,0.000242,0.000242,0.000229,0,15.0,-9.875995e-07,1.362607e-13,...,2.567491e-11,2.567491e-11,3.819802e-11,3.819802e-11,4.428414e-11,4.428414e-11,4.333109e-11,4.333109e-11,4.256766e-12,4.256766e-12


In [59]:
df_bands_2 = df_bands.drop(["participant_id"], axis=1)

In [60]:
X = df_bands_2[df_bands_2.columns.difference(["label"])]
y = df_bands_2["label"]

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(452, 19) (452,) (223, 19) (223,)


In [62]:
model = SVC()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(list(predictions))
print(list(y_test))

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1]
[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [63]:
score_of_the_model = model.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.48878923766816146


In [64]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.484304932735426


In [65]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.484304932735426


In [66]:
clf = make_pipeline(StandardScaler(), ExtraTreeClassifier())
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.5067264573991032


In [67]:
clf = make_pipeline(StandardScaler(), tree.DecisionTreeClassifier(max_depth=None, random_state=42))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.4663677130044843


In [68]:
clf = make_pipeline(StandardScaler(), LGBMClassifier(n_estimators=2000, random_state=0))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.49327354260089684


In [69]:
clf = make_pipeline(StandardScaler(), XGBClassifier(n_estimators=1000, random_state=0))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

model score: 0.49327354260089684
